In [ ]:
from bs4 import BeautifulSoup
import pandas as pd 
import os

def get_name_price(html):
    data = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    tables = soup.find_all('div', class_='pt-2')

    relevant = tables[0]

    listitems = relevant.find_all('li')

    for li in listitems:

        seller_names = li.find_all('span', attrs={'data-qa': 'seller-name'})
        seller_price = li.find_all('span', attrs={'data-qa': 'seller-price'})

        if seller_names and seller_price:
            data.append([seller_names[0].text, seller_price[0].text])

    data = pd.DataFrame(data)
    data.columns = ['pharmacy', 'price']
    return data




In [ ]:
fnames = [fname for fname in os.listdir('.') if fname.endswith('GoodRx.html')]

results = []

for fname in fnames:
    with open(fname, 'r') as f:
        html_content = f.read()
    data = get_name_price(html_content)
    drug_name = fname.split(' ')[0]
    data['name'] = drug_name
    results.append(data)

all_data = pd.concat(results)

all_data.to_csv('all_data.csv', index=False)